In [52]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer()

In [53]:
df1=pd.read_csv('books_cleaned.csv')

In [69]:
df1.describe()

,book_id,isbn13,original_publication_year,average_rating,ratings_count,score
count,7649.000000,7.649000e+03,7649.000000,7649.000000,7.649000e+03,7649.000000
mean,4707.302915,9.774524e+12,1990.129167,3.994470,6.200599e+04,3.997803
std,2890.277089,2.429361e+11,46.225576,0.250833,1.772387e+05,0.174395
min,1.000000,1.951703e+08,1308.000000,2.470000,2.773000e+03,2.790401
25%,2161.000000,9.780316e+12,1990.000000,3.840000,1.432700e+04,3.894867
50%,4582.000000,9.780451e+12,2004.000000,4.010000,2.302300e+04,4.004401
75%,7154.000000,9.780812e+12,2010.000000,4.170000,4.673100e+04,4.111017
max,9999.000000,9.790008e+12,2017.000000,4.820000,4.780653e+06,4.699789


In [55]:
df1.loc[(df1.original_publication_year<1000),'original_publication_year']=np.nan
df1=df1.dropna()


In [56]:
C=df1['average_rating'].mean()
print(C)
m=df1['ratings_count'].quantile(0.1)
m

3.9944698653418746


10835.0

In [57]:
def WR(x):
    v=x['ratings_count']
    R=x['average_rating']
    return (v/(v+m)*R) + (m/(m+v)*C)

In [58]:
df1['score'] = df1.apply(WR,axis=1)
df1.describe()

,book_id,isbn13,original_publication_year,average_rating,ratings_count,score
count,7649.000000,7.649000e+03,7649.000000,7649.000000,7.649000e+03,7649.000000
mean,4707.302915,9.774524e+12,1990.129167,3.994470,6.200599e+04,3.997803
std,2890.277089,2.429361e+11,46.225576,0.250833,1.772387e+05,0.174395
min,1.000000,1.951703e+08,1308.000000,2.470000,2.773000e+03,2.790401
25%,2161.000000,9.780316e+12,1990.000000,3.840000,1.432700e+04,3.894867
50%,4582.000000,9.780451e+12,2004.000000,4.010000,2.302300e+04,4.004401
75%,7154.000000,9.780812e+12,2010.000000,4.170000,4.673100e+04,4.111017
max,9999.000000,9.790008e+12,2017.000000,4.820000,4.780653e+06,4.699789


In [59]:
def lng(x,str1):
    if (x['language_code'] == str1):
        return x

In [60]:
def clean_data(x):
    if isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return ''

In [61]:
df1['authors'] = df1['authors'].apply(clean_data)

In [62]:
def create_soup(x):
    return ' ' + x['authors'] + ' ' +  x['description']
df1['soup'] = df1.apply(create_soup, axis=1)

In [63]:
df1['original_publication_year'].describe()

count    7649.000000
mean     1990.129167
std        46.225576
min      1308.000000
25%      1990.000000
50%      2004.000000
75%      2010.000000
max      2017.000000
Name: original_publication_year, dtype: float64

In [64]:
#df1 = df1.reset_index()
#indices = pd.Series(df1.index, index=df1['original_title'])

In [65]:
df1.head(5)

,book_id,isbn,isbn13,authors,original_publication_year,original_title,language_code,average_rating,ratings_count,description,score,soup
0,1,439023483,9.780439e+12,suzannecollins,2008.0,The Hunger Games,eng,4.34,4780653,Winning will make you famous. Losing means cer...,4.339219,suzannecollins Winning will make you famous. ...
1,2,439554934,9.780440e+12,"j.k.rowling,marygrandpré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479,Harry Potter's life is miserable. His parents ...,4.438954,"j.k.rowling,marygrandpré Harry Potter's life ..."
2,3,316015849,9.780316e+12,stepheniemeyer,2005.0,Twilight,en-US,3.57,3866839,About three things I was absolutely positive.F...,3.571186,stepheniemeyer About three things I was absol...
3,4,61120081,9.780061e+12,harperlee,1960.0,To Kill a Mockingbird,eng,4.25,3198671,The unforgettable novel of a childhood in a sl...,4.249137,harperlee The unforgettable novel of a childh...
4,5,743273567,9.780743e+12,f.scottfitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664,"THE GREAT GATSBY, F. Scott Fitzgerald’s third ...",3.890420,"f.scottfitzgerald THE GREAT GATSBY, F. Scott ..."


In [66]:
df1.shape

(7649, 12)

In [67]:
def get_recommendations(titles):
    # Get the index of the movie that matches the title
    df3=df1
    scr=5
    df2=pd.DataFrame()
    all_titles=""
    all_soup=""
    for title in titles:
        lang=df1['language_code'][df1.original_title == title].values[0]
                ##################### CONCATENATION DONE#################################################
        df2=df2.append(df3[df3.language_code == lang])
        df3=df3[df3.language_code !=lang]
        scr_temp=int(df1['score'][df1.original_title == title].values[0])
        all_titles+=title+" "
        all_soup+=df1.loc[(df1.original_title==title),'soup'].values[0] + " "
        if(scr_temp<scr):
            scr=scr_temp
    scr=scr - 0.5
    df2=df2[df2.score>scr]
    print
    for title in titles:
        i = df2[(df2.original_title==title)].index.values[0]
        df2 = df2.drop(i)
    df2 = df2.append({'book_id':0000, 'authors':'not required', 'original_publication_year':20, 'original_title':all_titles,
       'title':all_titles, 'language_code':lang, 'average_rating':0, 'ratings_count':0000,'score':scr,'soup':all_soup} , ignore_index=True)
    count_matrix = count.fit_transform(df2['soup'])
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
    df2 = df2.reset_index()
    indices = pd.Series(df2.index, index=df2['original_title'])
    idx = indices[all_titles]
    # Get the pairwsie similarity scores of all movies with that movie
    #C_M(lang)
    sim_scores = list(enumerate(cosine_sim2[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    #print(sim_scores)
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return df2['original_title'].iloc[movie_indices].values

In [68]:
z=(get_recommendations(['Gone Girl']))
print('Here you go')
for a in z:
    print(a)

Here you go
That Old Cape Magic
Prince of Thorns
The Crossing 
The Wolf Gift
Shantaram
Captain Blood: His Odyssey
Tigers in Red Weather
Thomas Jefferson: The Art of Power
Matterhorn: A Novel of the Vietnam War
Anathem


In [31]:
get_recommendations(['Gone Girl','Dark Places'])

[(5900, 0.9999999999999998), (168, 0.7071067811865475), (1430, 0.7071067811865475), (5582, 0.1414213562373095), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0)]


5900            Gone Girl To Kill a Mockingbird 
168                                  Dark Places
1430                             What Do You Do?
5582                                      Rogues
0                               The Hunger Games
1       Harry Potter and the Philosopher's Stone
2                               The Great Gatsby
3                         The Fault in Our Stars
4                         The Catcher in the Rye
5                            Pride and Prejudice
6                               The Kite Runner 
Name: original_title, dtype: object